In [1]:
import comment_lib
import seaborn as sns; sns.set()
import pandas as pd
import itertools
import csv
import random

%matplotlib inline

comments_file = 'comments_nl_right.csv'
videos_file = 'videos_nl_right.csv'
channel_file = 'channels_nl_right.csv'

## Network edges for gephi

In [3]:
comment_lib.generate_network_comments_aggregated(data,'edges_comments_videonodes_agg.csv','video', minimum_connections = 3)

## Comment descriptives

This notebook serves to supply descriptives for the comments data. Common manipulation methods are drawn from the comment_lib.py file supplied on GitHub.

In [2]:
with open(comments_file, 'r', encoding = 'utf8') as f:
    data = comment_lib.read_comments(f)

data = comment_lib.add_video_metadata(data, videos_file, channel_file)

## Number of commenters

1. Are commenters serial commenters or do they stick to their own thing?

In [ ]:
commenter_groups = data.groupby('author_channel_id')

#### GINI plot commenters

Comment 'ownership', inequality?

In [ ]:
num_comments = pd.DataFrame(commenter_groups.size().sort_values(ascending = True), columns = ['count'])
num_comments['Cumulative percentage of comments'] = 100*num_comments['count'].cumsum()/max(num_comments['count'].cumsum())
num_comments['Commenter percentile'] = num_comments.reset_index().index/max(num_comments.reset_index().index)

sns.lineplot(x=num_comments['Commenter percentile'],y=num_comments['Cumulative percentage of comments'])

print('20% van de commenters plaatst 70% van de comments en de top-10% plaatst 60% van de comments')
del num_comments

#### Number of channels per commenter

In [ ]:
commenter_groups['channel_title'].nunique().value_counts()

## Comment length

Variabelen:
- comment_length_chars : Aantal characters (simpel: len)
- comment_length_words : Aantal woorden, ruw bepaald met splitten op spatie.

In [ ]:
data['comment_length_chars'] = data['comment_text'].apply(len)
data['comment_length_words'] = data['comment_text'].apply(lambda x: len(x.split()))

In [ ]:
data[['comment_length_words','comment_length_chars']].describe()

In [ ]:
#Plot variabele voor de lengte
# - Cutoff kan gekozen worden om de plot leesbaar te maken. Kies cutoff erg hoog (e.g. 10**6) om te zien hoe het zonder is

var = 'comment_length_words'
plot_cutoff = 250
bin_count = 100

sns.distplot(data[var][data[var] < plot_cutoff], bins = bin_count)

Het aantal woorden is relatief klein (mediaan 11) met grote uitschieters.

In [ ]:
chars = data['comment_length_chars']
print('{:0.2%} van de comments past ook binnen een tweet.'.format(len(chars[chars <= 280])/len(chars)))
del chars

In [ ]:
print('Om te kijken naar aantal referenties naar andere commenters/gebruikers:\n')

comments = data['comment_text']
print('{:0.2%} van de comments bevat een "@"'.format(len(comments[comments.str.contains('@')])/len(comments)))
del comments
print('\n"@" betekent doorgaans een referentie naar een andere gebruiker (zie volgende cel voor test)')

In [ ]:
comments = data['comment_text']
comments = comments[comments.str.contains("@")]

for comment in random.sample(list(comments),10):
    print(comment)
    print('==================')
del comments

In [ ]:
data.comment_id.apply(lambda x: x[0]).value_counts()

In [ ]:
data[data['video_id'] == '__Ou7gS9F38']